# Version 3 took from Github and now tring to integrate W&B with this - Kasundie

In [1]:
!pip install transformers datasets rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.0 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=4d4dd1311f4705a99f49cbd3009ba3cd2a3a217f9e0ec719736298e4d701ddf2
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [2]:
pip install wandb

In [ ]:
# !pip install nltk sacrebleu
# import nltk
# from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
# from rouge_score import rouge_scorer
# from sacrebleu.metrics import BLEU, CHRF, TER
# nltk.download('wordnet')  # Required for METEOR

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kasundiehewawasam (kasundiehewawasam-uop) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [4]:
import wandb
from datetime import datetime

# Initialize a new W&B run
run_name = f"t5-small-{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
wandb.init(project="text-summarizer",
           name=run_name,
           config={
              "model": "t5-small",
              "max_length": 100,
              "num_beams": 5,
              "length_penalty": 1.2,
              "temperature": 0.9})


wandb: Currently logged in as: kasundiehewawasam (kasundiehewawasam-uop) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [5]:
from datasets import load_dataset
from rouge_score import rouge_scorer
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import numpy as np

In [6]:
# 1. Load a VERY Small Subset of the Dataset
dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:1%]")  # Use only 1% of the training data

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [8]:
dataset_test = load_dataset("cnn_dailymail", "3.0.0", split="test[:1%]")  # Small test set for evaluation

In [9]:
eference_summary = dataset_test["highlights"]

In [10]:
# 2. Choose a Smaller Model
model_name = 't5-small'  # Can upgrade to 't5-base' for better quality
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [11]:
# 3. Preprocess Data (Optimized for Abstractive Summarization)
def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length", return_tensors="pt")

    labels = tokenizer(examples["highlights"], max_length=150, truncation=True, padding="max_length", return_tensors="pt")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)
tokenized_datasets_test = dataset_test.map(preprocess_function, batched=True)


Map:   0%|          | 0/2871 [00:00<?, ? examples/s]

Map:   0%|          | 0/115 [00:00<?, ? examples/s]

In [13]:
# 4. Training (Optimized for Faster Learning)
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",
    learning_rate=3e-5,  # Slightly increased learning rate
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    save_steps=1000,
    logging_steps=100,
    push_to_hub=False,
    report_to="wandb"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets_test,  # Added evaluation dataset
    tokenizer=tokenizer,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-13-b84751309743>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [14]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
100,3.609600
200,1.097600
300,1.031000


TrainOutput(global_step=359, training_loss=1.7708002265781413, metrics={'train_runtime': 39.661, 'train_samples_per_second': 72.388, 'train_steps_per_second': 9.052, 'total_flos': 388566312026112.0, 'train_loss': 1.7708002265781413, 'epoch': 1.0})

In [15]:
# 5. Inference (Abstractive Summarization)
def generate_summary(text):
    input_ids = tokenizer.encode("summarize: " + text, return_tensors="pt").to(model.device)
    output = model.generate(
        input_ids,
        max_length=100,
        num_beams=5,  # More beams for better summarization
        early_stopping=True,
        length_penalty=1.2,  # Encourages slightly shorter summaries
        temperature=0.9  # Encourages more diverse output
    )
    summary = tokenizer.decode(output[0], skip_special_tokens=True)

    # Log metrics to W&B
    wandb.log({
        "input_length": len(text),
        "output_length": len(summary),
        "generated_summary": summary
    })

    return summary

In [16]:
# 6. Model Evaluation - ROUGE Score Calculation
def evaluate_model(test_dataset):
    rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge1_scores, rouge2_scores, rougeL_scores = [], [], []

    for i, example in enumerate(test_dataset):
        text = example["article"]
        reference_summary = example["highlights"]

        # Generate model's summary
        generated_summary = generate_summary(text)

        # Compute ROUGE scores
        scores = rouge.score(reference_summary, generated_summary)

        rouge1_scores.append(scores["rouge1"].fmeasure)
        rouge2_scores.append(scores["rouge2"].fmeasure)
        rougeL_scores.append(scores["rougeL"].fmeasure)

        if i < 5:  # Print first 5 summaries for reference
            print(f"\nExample {i+1}:")
            print(f"Original Text: {text[:300]}...")  # Show only first 300 characters
            print(f"Reference Summary: {reference_summary}")
            print(f"Generated Summary: {generated_summary}")

    # Compute average ROUGE scores
    avg_rouge1 = np.mean(rouge1_scores)
    avg_rouge2 = np.mean(rouge2_scores)
    avg_rougeL = np.mean(rougeL_scores)

    print("\n==== Model Evaluation Results ====")
    print(f"ROUGE-1 Score: {avg_rouge1:.4f}")
    print(f"ROUGE-2 Score: {avg_rouge2:.4f}")
    print(f"ROUGE-L Score: {avg_rougeL:.4f}")

    wandb.log({
    "Evaluation/ROUGE-1": avg_rouge1,
    "Evaluation/ROUGE-2": avg_rouge2,
    "Evaluation/ROUGE-L": avg_rougeL
})


In [17]:
# Run evaluation
evaluate_model(dataset_test)

Token indices sequence length is longer than the specified maximum sequence length for this model (789 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



Example 1:
Original Text: (CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the cou...
Reference Summary: Membership gives the ICC jurisdiction over alleged crimes committed in Palestinian territories since last June .
Israel and the United States opposed the move, which could open the door to war crimes investigations against Israelis .
Generated Summary: "Indeed, today brings us closer to our shared goals of justice," Palestinian Foreign Minister says. Israel and the United States oppose the Palestinians' attempts to join the Rome Statute. The International Criminal Court was set up in 2002 to prosecute genocide and crimes against humanity. Palestinians signed the ICC's founding Rome Statute in January.

Example 2:
Original Text: (CNN)Never min

In [18]:
# Save model & tokenizer
print("Saving model and tokenizer...")
model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")

wandb.save("./saved_model/*")  # Save model to W&B
# wandb.save("./saved_model/pytorch_model.bin")
# wandb.save("./saved_model/config.json")
# wandb.save("./saved_model/tokenizer_config.json")
print("Model saved successfully!")

Saving model and tokenizer...


wandb: WARNING Symlinked 7 files into the W&B run directory, call wandb.save again to sync new files.


Model saved successfully!


In [ ]:
# # 6. Examples
# texts = [
#     "Climate change is increasingly affecting global agriculture, posing significant challenges to food security and livelihoods. Rising temperatures, altered precipitation patterns, and increased frequency of extreme weather events are some of the key factors impacting crop yields and livestock productivity. For instance, prolonged droughts can lead to water scarcity, affecting irrigation and reducing crop growth. Similarly, excessive rainfall can cause flooding, damaging crops and soil health.",

#     "The Sri Lankan government said on Saturday that the recent killings in the country, including that of a prominent underworld figure, appear to be an attempt to undermine the ongoing investigations into high-profile criminal cases such as the 2019 Easter Sunday suicide bombings. The government’s response came following this week’s courtroom shooting that left the underworld figure dead.  The incident sent shockwaves across the country, leading to the opposition probing the government’s measures to tackle the spate of violence."
# ]



In [ ]:
# for i, text in enumerate(texts):
#     print(f"\n**Original Text {i+1}:**\n{text}\n")
#     print(f"**Generated Summary {i+1}:**\n{generate_summary(text)}\n")


In [19]:
# Finish W&B logging
wandb.finish()

Evaluation/ROUGE-1,▁
Evaluation/ROUGE-2,▁
Evaluation/ROUGE-L,▁
input_length,▄▁▅▄▂▅▂▄▃▂▃▂█▂▄▆▂▂▄▂▁▁▅▂▂▂▂▁▂▃▄▂▂▆▂▂▅▄▃▁
output_length,▄▅█▆▆▃▆▇▅▅▇▅▅▆▆▆▅▅▇▄▃▄▄▆▁▄▆▂▅█▃▇▅▄▅▄▅▅▅▅
train/epoch,▁▄▆█
train/global_step,▁███████████████████████████████████████
train/grad_norm,█▂▁
train/learning_rate,█▄▁
train/loss,█▁▁
Evaluation/ROUGE-1,0.26398
